# 1. Prerequisites: Bayes' Rule & Notation

## Standard Bayes' Rule
$$
P(A \mid B) = \frac{P(B \mid A)\,P(A)}{P(B)}
$$

## Bayes' Rule with Background Knowledge (Context)
In robotics, we always have past context $C$. Bayes' rule with context adds $C$ to all terms:
$$
P(A \mid B, C) = \frac{P(B \mid A, C)\,P(A \mid C)}{P(B \mid C)}
$$

## Sequence Notation
We use colon notation to represent a sequence:
$$
z_{1:t} = \{z_1, z_2, \ldots, z_t\}
$$

Joint intersection:
$$
P(z_1 \cap z_2 \cap \cdots \cap z_t \mid A) = P(z_{1:t} \mid A)
$$

---

# 2. The Goal: Recursive Bayes Filter

We want to represent the belief distribution of the state $x_t$ recursively, without showing a explicit solution:

$$
\text{bel}(x_t) = f(\text{bel}(x_{t-1}),\; z_t,\; u_t)
$$

Definition of belief:
$$
\text{bel}(x_t) = p(x_t \mid z_{1:t}, u_{1:t})
$$

Split the measurement history:
$$
\text{bel}(x_t) = p(x_t \mid z_t, z_{1:t-1}, u_{1:t})
$$

---

# 3. Applying Bayes' Rule

Let:

- $A = x_t$  
- $B = z_t$  
- $C = \{z_{1:t-1}, u_{1:t}\}$

Applying Bayes:
$$
p(x_t \mid z_t, z_{1:t-1}, u_{1:t})
=
\frac{
p(z_t \mid x_t, z_{1:t-1}, u_{1:t}) \;
p(x_t \mid z_{1:t-1}, u_{1:t})
}{
p(z_t \mid z_{1:t-1}, u_{1:t})
}
$$

## Normalizer (η)

The denominator does not depend on $x_t$. Replace with a constant:

$$
\eta = \frac{1}{p(z_t \mid z_{1:t-1}, u_{1:t})}
$$

Thus:

$$
\text{bel}(x_t)
=
\eta \;
p(z_t \mid x_t, z_{1:t-1}, u_{1:t})
\;
p(x_t \mid z_{1:t-1}, u_{1:t})
$$

---
# 4. The Markov Assumption

Under the **Markov Assumption**, the sensor measurement at time $t$ depends **only** on the current state $x_t$, not on any past measurements or controls:

$$
p(z_t \mid x_t, z_{1:t-1}, u_{1:t}) \approx p(z_t \mid x_t)
$$

Substituting this into the Bayes update equation:

$$
\text{bel}(x_t)
=
\eta \;
p(z_t \mid x_t)
\;
p(x_t \mid z_{1:t-1}, u_{1:t})
$$

This can be written as:

$$
\text{bel}(x_t) = \eta \;
\underbrace{p(z_t \mid x_t)}_{\text{Measurement Model}}
\;
\underbrace{p(x_t \mid z_{1:t-1}, u_{1:t})}_{\text{Prediction (Prior)}}
$$

---

We now focus on the *Prediction (Prior)* term:

$$
\underbrace{p(x_t \mid z_{1:t-1}, u_{1:t})}_{\text{Prediction (Prior)}}
$$

### Why this term is difficult

This term tries to answer:

**“Where am I now, given all past measurements and controls?”**

But the robot’s state does not appear out of nowhere.  
To reach the current state $x_t$, the robot must have come from **some** previous state $x_{t-1}$.

### How to handle it

We use the **Law of Total Probability** and sum (integrate) over every possible previous state:

$$
p(x_t \mid z_{1:t-1}, u_{1:t})
=
\int
\underbrace{p(x_t \mid x_{t-1}, z_{1:t-1}, u_{1:t})}_{\text{Transition Probability}}
\;
\underbrace{p(x_{t-1} \mid z_{1:t-1}, u_{1:t})}_{\text{Previous Belief}}
\; dx_{t-1}
$$

In words:

- The **transition model** tells us how likely it is to move from $x_{t-1}$ to $x_t$ given controls.  
- The **previous belief** tells us how likely the robot was at $x_{t-1}$ before receiving the new measurement.

Together, they form the prediction step of the Bayes Filter.




## What we can notice so far on the Bayes Framework / Bayes Filter 
it is a recursive state estimation technique
different implementations for the exact solution of this recursive techniuqes based on the data proprties
- Linear Vs Non Linear Motion models will influence how we implement it-
- Gaussains Vs Non Guassians will also affect the implemntation
- Parametric Vs Non Parametric
  